# Kategoriale Variablen

Open in Colab: [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/febse/econ2025-de/blob/main/09-Categorical-Variables.ipynb)

## Simpson's paradox

Der Datensatz `berkley` enthält Informationen über Zulassungen an die University of California, Berkeley. Die Variablen sind:
- `Major`: Studienrichtung (A, B, C, etc.)
- `Gender`: Geschlecht (M, F)
- `Admission`: Zulassung (Accepted, Rejected)
- `is_rejected`: Eine Indikatorvariable, die angibt, ob eine Person abgelehnt wurde (1) oder nicht (0)


In [ ]:
library(tidyverse)

## Simpson's paradox (1)

berkley <- read_csv(
  "https://waf.cs.illinois.edu/discovery/berkeley.csv"
  ) %>%
  mutate(
    is_rejected = ifelse(Admission == "Accepted", 0, 1),
  )

berkley %>% head()

In [ ]:
berkley %>%
  group_by(Gender) %>%
    summarize(
        n = n(),
        n_rejected = sum(is_rejected),
        p_rejected = round(mean(is_rejected), 2)
    )

In [ ]:
admission_by_major_gender <- berkley %>%
  group_by(Major, Gender) %>%
    summarize(
        n = n(),
        n_rejected = sum(is_rejected),
        p_rejected = round(mean(is_rejected), 2)
    )
admission_by_major_gender

In [ ]:
admission_by_major_gender %>%
  ggplot(aes(x = Major, y = p_rejected, fill=Gender)) +
    geom_bar(stat = "identity", position = "dodge") +
    labs(
      y = "Admission rate",
      x = "Major"
    ) +
    theme(axis.text.x = element_text(angle = 45, hjust = 1))

## Regression with Categorical Variables



In [ ]:
berkley %>% head()

In [ ]:
model_matrix <- model.matrix(lm(is_rejected ~ 0 + Gender, data = berkley))
model_matrix %>% head()

In [ ]:
berkey <- berkley %>%
  bind_cols(model_matrix)
berkey %>% head()

Lassen Sie uns die Mittelwerte der Indikatorvariable für die Zulassung nach Geschlecht und Studienrichtung berechnen.

Das Modell ist wie folgt:

$$
\widehat{\text{is\_rejected}}_i = \hat{\beta}_0 + \hat{\beta}_{F} \text{GenderF}_i + \hat{\beta}_{M} \text{GenderM}_i  
$$

Eine alternative Art und Weise, das Modell zu schreiben, ist:

$$
\widehat{\text{is\_rejected}}_i = \hat{\beta}_0 + \hat{\beta}_{\text{Gender}[i]}
$$


In [ ]:
lm(is_rejected ~ Gender, data = berkley) %>%
  summary()

## BIP pro Kopf und Staatsausgaben

Der Datensatz `gdp2019` enthält die BIP pro Kopf in aktuellen Preisen für 142 Länder im Jahr 2019. Darüber hinaus zeigt die Variable `spending` die Staatsausgaben in diesen Ländern als Anteil (in Prozent) des BIP.

-   `gdppc` (numeric): [BIP pro Kopf](https://www.imf.org/external/datamapper/NGDPDPC@WEO/OEMDC/ADVEC/WEOWORLD) in USD (aktuelle Preise)
-   `spending` (numeric): [Staatsausgaben als Anteil des BIP](https://www.imf.org/external/datamapper/exp@FPP/USA/FRA/JPN/GBR/SWE/ESP/ITA/ZAF/IND).

In [ ]:
gdp2019 <- read_csv(
  "https://raw.githubusercontent.com/feb-sofia/econometrics-2023/main/data/gdpgov2019.csv"
  ) %>%
  filter(!is.na(spending))

gdp2019 %>% head()

1.  Zuerst möchten wir uns die Daten anschauen

$$
\text{gdppc}_i = \beta_0 + \beta_1 \text{spending}_i + e_i, \quad e_i \sim N(0, \sigma^2)
$$


In [ ]:
gdp2019 %>%
  ggplot(aes(x = spending, y = gdppc)) +
  geom_point() +
  geom_smooth(method = "lm") +
  labs(
    y = "GDP per capita",
    x = "Gov. spending"
  ) +
  lims(
    x = c(0, 70),
    y = c(0, 90000)
  )

In [ ]:
# Estimate the model


2.  Nun möchten wir die Daten in Kategorien einteilen. Wir verwenden die folgende Einteilung:

-   Low: if $\text{gdppc} \leq 1025$
-   Medium-low: if $1025 < \text{gdppc} \leq 3995$
-   Medium-high: if $3995 < \text{gdppc} \leq 12375$
-   High: if $12375 < \text{gdppc} \leq 30000$
-   Very high: if $\text{gdppc} > 30000$

In [ ]:
gdp2019 <- gdp2019 %>%
  mutate(
    gdppc_gr = cut(
      gdppc,
      breaks = c(0, 1025, 3995, 12375, 30000, Inf),
      labels = c("Low", "Medium-low", "Medium-high", "High", "Very high"))
  )
table(gdp2019$gdppc_gr)

1. Wir schätzen zuerst ein lineares Modell mit den Kategorien als Dummy-Variablen:

$$
\text{gdppc}_i = \beta_0 + \beta_{\text{gdppc\_gr}[i]} + e_i, e_i \sim N(0, \sigma^2)
$$

Dabei ist:

$$
\beta_{\text{gdppc\_gr}[i]} = \begin{cases}
\beta_{\text{low}} & \text{falls } \text{gdppc\_gr}_i = \text{Low} \\
\beta_{\text{medium-low}} & \text{falls } \text{gdppc\_gr}_i = \text{Medium-low} \\
\beta_{\text{medium-high}} & \text{falls } \text{gdppc\_gr}_i = \text{Medium-high} \\
\beta_{\text{high}} & \text{falls } \text{gdppc\_gr}_i = \text{High} \\
\beta_{\text{very-high}} & \text{falls } \text{gdppc\_gr}_i = \text{Very high}
\end{cases}
$$

Eine andere Art und Weise, das Modell aufzuschreiben, ist:

$$
\text{gdppc}_i = \beta_0 + \beta_1 \text{low}_i + \beta_2 \text{medium-low}_i + \beta_3 \text{medium-high}_i + \beta_4 \text{high}_i + \beta_5 \text{very-high}_i + e_i, e_i \sim N(0, \sigma^2)
$$



In [ ]:
gdp2019 %>%
  ggplot(aes(x = spending, y = gdppc)) +
  geom_point(aes( color = gdppc_gr)) +
  geom_smooth(method = "lm") +
  # geom_abline(
  #   intercept = c(-144.39, 1308.32, 5835.94, 18460.21, 51920.69),
  #   slope = 44.80,
  #   alpha = 0.5
  # ) +
  lims(
    x = c(0, 80),
    y = c(0, 90000)
  )

## F-Test

Very often we want test the hypothesis that a certain subset of the coefficients are simultaneously equal to zero. This is called a joint hypothesis.

In our example we could test the hypothesis that the coefficients of `Med Low`, `Med High`, `High` and `Very High` are all equal to zero. This is equivalent to testing if the average of `gdppc` is the same in all groups.

The test is called an F-test and is based on the ratio of the sum of squared errors of the restricted model and the sum of squared errors of the unrestricted model.

The restricted model is the one where the coefficients of `Med Low`, `Med High`, `High` and `Very High` are all equal to zero.

The unrestricted model is the one where all coefficients are estimated.

The F-test is based on the following statistic:

$$
F = \frac{(RSS_{\text{restricted}} - RSS_{\text{unrestricted}}) / (p_{\text{restricted}} - p_{\text{unrestricted}})}{RSS_{\text{unrestricted}} / (n - p_{\text{unrestricted}})}
$$

$$
F = \frac{(RSS_{\text{restricted}} - RSS_{\text{unrestricted}})}{RSS_{\text{unrestricted}}}\frac{(n - p_{\text{unrestricted}})}{(p_{\text{restricted}} - p_{\text{unrestricted}})}
$$

where $RSS$ is the residual sum of squares, $p$ is the number of parameters in the model and $n$ is the number of observations.

## Interaction Effects


In [ ]:
# Load the data
stu <- read.csv("https://raw.githubusercontent.com/febse/data/main/econ/drinking.csv") %>%
  mutate(
    sex = ifelse(male == 1, "Male", "Female")
  )

# Print the first few rows
stu %>% head()

In [ ]:
stu %>%
  ggplot(aes(x=gpa, y=drink)) +
    geom_point() +
    geom_smooth(method=lm)

Estimate and iterpret the results of the following models:

- $\text{drink}_i = \beta_0 + \beta_1\text{gpa}_i + e_i$
- $\text{drink}_i = \beta_0 + \beta_1 \text{sexMale}_i + e_i$
- $\text{drink}_i =\beta_0 + \beta_1 \text{gpa}_i + \beta_2 \text{sexMale}_i + e_i$
- $\text{drink}_i = \beta_0 + \beta_1 \text{gpa}_i + \beta_2 \text{sexMale}_i +
\beta_3 \text{gpa}_i \cdot \text{sexMale}_i$

## Polynomial regression

- Sample 100 observations from the following model. Assume that $x_i$ is uniformly distributed between -4 and 3.

$$
y_i = 2 + x_i + x^2 + e_i, e_i \sim N(0, 1)
$$

- Fit a linear regression model to the data, plot the data in a scatterplot and add the estimated regression line.
- Calculate the residuals and plot them against the fitted values. What do you observe?



In [ ]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

In [ ]:
## Create the simulated sample
sim_n <- 100

poly_dt <- tibble(
  x = runif(n = sim_n, min = -4, max = 3),
  y = 2 +  x +  x^2 + rnorm(n = sim_n, mean = 0, sd = 1)
)

poly_dt %>%
  ggplot(aes(x = x, y = y)) +
  geom_point()



2.  Estimate a linear model for `x` and `y`

$$
y_i = \beta_0 + \beta_1 x_i, e_i \sim N(0, \sigma^2)
$$


In [ ]:
# fit1_pred <- predict(
#   fit1,
#   interval = "confidence"
#   ) %>%
#   as_tibble() %>%
#   bind_cols(poly_dt) %>%
#   mutate(
#     res = y - fit
#   )

In [ ]:
# fit1_pred %>%
#   ggplot(aes(x = fit, y = res)) +
#   geom_point() +
#   geom_hline(yintercept = 0, alpha = 0.5, lty = 2) +
#   labs(
#     x = "Predicted",
#     y = "Residual"
#   )


3.  Add a quadratic term to the linear model and estimate it

$$
y_i = \beta_0 + \beta_1 x_i + \beta_2 x^2_i + e_i, e_i \sim N(0, \sigma^2)
$$


In [ ]:
# fit2_pred <- predict(fit2, interval = "confidence") %>%
#   as_tibble() %>%
#   bind_cols(poly_dt) %>%
#   mutate(
#     res = y - fit
#   )

In [ ]:
# fit2_pred %>%
#   ggplot(aes(x = x, y = y)) +
#   geom_point() +
#   geom_line(
#     aes(y = fit),
#     color = "steelblue2"
#   ) +
#   geom_ribbon(
#     aes(ymin = lwr, ymax = upr),
#     alpha = 0.2,
#     fill = "steelblue2"
#   )

In [ ]:
# fit2_pred %>%
#   ggplot(aes(x = fit, y = res)) +
#   geom_point() +
#   geom_hline(yintercept = 0, alpha = 0.5, lty = 2) +
#   labs(
#     x = "Predicted",
#     y = "Residual"
#   )

4.  Add the third and fourth degree polynomial terms to the model and estimate it

$$
y_i = \beta_0 + \beta_1 x_i + \beta_2 x^2_i + \beta_3 x^3_i + \beta_4 x^4_i + e_i, e_i \sim N(0, \sigma^2)
$$

In [ ]:
# fit3_pred <- predict(fit3, interval = "confidence") %>%
#   as_tibble() %>%
#   bind_cols(poly_dt) %>%
#   mutate(
#     res = y - fit
#   )

In [ ]:
# fit3_pred %>%
#   ggplot(aes(x = x, y = y)) +
#   geom_point() +
#   geom_line(
#     aes(y = fit),
#     color = "steelblue2"
#   ) +
#   geom_ribbon(
#     aes(ymin = lwr, ymax = upr),
#     alpha = 0.2,
#     fill = "steelblue2"
#   )

In [ ]:
# fit3_pred %>%
#   ggplot(aes(x = fit, y = res)) +
#   geom_point() +
#   geom_hline(yintercept = 0, alpha = 0.5, lty = 2) +
#   labs(
#     x = "Predicted",
#     y = "Residual"
#   )

## Model choice

1.  $R^2$

$$
R^2 = 1 - \frac{RSS}{TSS} \\
RSS = \sum_{i = 1}^{n}(y_i - \hat{y}_i)^2 \\
TSS = \sum_{i = 1}^{n}(y_i - \bar{y})^2
$$

Compute $R^2$ for the three models. Which model fits the data best according to it?


In [ ]:
sim_noconst <- tibble(
  y = 2 + rnorm(n = 100, mean = 0, sd = 1),
  x = runif(n = 100)
)

sim_noconst %>%
  ggplot(aes(x = x, y = y)) +
  geom_point()

2.  Adjusted $R^2$

$$
R^2_{ADJ} = 1 - \frac{n - 1}{n - p - 1} \frac{RSS}{TSS}
$$

3.  Information Criteria

Compute the Akaike information criterion (AIC) for the three models and choose the best one.

$$
\text{AIC} = n \log\left(\frac{RSS}{n}\right) + 2p + n + n \log(2\pi).
$$

In [ ]:
AIC(fit1)
AIC(fit2)
AIC(fit3)

In [ ]:
smpl <- sample(1:100, 20)
poly_dt_estim <- poly_dt[smpl, ]
poly_dt_test <- poly_dt[-smpl, ]

fit2_estim <- lm(y ~ poly(x, 4), data = poly_dt_estim)
fit5_test <- lm(y ~ poly(x, 4), data = poly_dt_test)

pred2 <- predict(fit2_estim, newdata = poly_dt_estim)
pred5 <- predict(fit2_estim, newdata = poly_dt_estim)